# Data cleaning and feature creation
Often writes data to data/processed or data/interim

In [1]:
import polars as pl
import os

DATA_PATH = "../data/raw"

In [2]:
# Load data
file_names = [x for x in os.listdir(DATA_PATH) if ".csv" in x]  # List all CSV files in the raw data directory

df_list = []
for file in file_names:
    print("Loading file:", file)
    df = pl.scan_csv(os.path.join(DATA_PATH, file), infer_schema=False)
    df_list.append(df)
print("Loaded", len(df_list), "files.")

Loading file: ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv
Loading file: ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv
Loading file: ResaleFlatPricesBasedonRegistrationDateFromMar2012toDec2014.csv
Loaded 3 files.


In [3]:
# Combine lazyframes into a single lazyframe
df_combined = pl.concat(df_list, how="align_full")
df_combined_col = df_combined.collect()

In [4]:
# Convert datatype
df_combined_col.head()

month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
str,str,str,str,str,str,str,str,str,str,str
"""2012-03""","""ANG MO KIO""","""2 ROOM""","""172""","""ANG MO KIO AVE 4""","""06 TO 10""","""45""","""Improved""","""1986""",null,"""250000"""
"""2012-03""","""ANG MO KIO""","""2 ROOM""","""510""","""ANG MO KIO AVE 8""","""01 TO 05""","""44""","""Improved""","""1980""",null,"""265000"""
"""2012-03""","""ANG MO KIO""","""3 ROOM""","""103""","""ANG MO KIO AVE 3""","""06 TO 10""","""73""","""New Generation""","""1978""",null,"""368000"""
"""2012-03""","""ANG MO KIO""","""3 ROOM""","""110""","""ANG MO KIO AVE 4""","""01 TO 05""","""67""","""New Generation""","""1978""",null,"""323000"""
"""2012-03""","""ANG MO KIO""","""3 ROOM""","""114""","""ANG MO KIO AVE 4""","""01 TO 05""","""73""","""New Generation""","""1978""",null,"""339000"""


In [5]:
# Convert variables to appropriate types
df_combined_col = df_combined_col.with_columns(
    pl.col("resale_price").cast(pl.Float64),
    pl.col("lease_commence_date").cast(pl.Int64),
    pl.col("floor_area_sqm").cast(pl.Float64),
    pl.col("month").str.to_date("%Y-%m")
)
df_combined_col.head()

month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
date,str,str,str,str,str,f64,str,i64,str,f64
2012-03-01,"""ANG MO KIO""","""2 ROOM""","""172""","""ANG MO KIO AVE 4""","""06 TO 10""",45.0,"""Improved""",1986,null,250000.0
2012-03-01,"""ANG MO KIO""","""2 ROOM""","""510""","""ANG MO KIO AVE 8""","""01 TO 05""",44.0,"""Improved""",1980,null,265000.0
2012-03-01,"""ANG MO KIO""","""3 ROOM""","""103""","""ANG MO KIO AVE 3""","""06 TO 10""",73.0,"""New Generation""",1978,null,368000.0
2012-03-01,"""ANG MO KIO""","""3 ROOM""","""110""","""ANG MO KIO AVE 4""","""01 TO 05""",67.0,"""New Generation""",1978,null,323000.0
2012-03-01,"""ANG MO KIO""","""3 ROOM""","""114""","""ANG MO KIO AVE 4""","""01 TO 05""",73.0,"""New Generation""",1978,null,339000.0


In [6]:
# Check for nulls
null_counts = df_combined_col.null_count()
print("Null counts in each column:")
print(null_counts)

Null counts in each column:
shape: (1, 11)
┌───────┬──────┬───────────┬───────┬───┬────────────┬───────────────┬───────────────┬──────────────┐
│ month ┆ town ┆ flat_type ┆ block ┆ … ┆ flat_model ┆ lease_commenc ┆ remaining_lea ┆ resale_price │
│ ---   ┆ ---  ┆ ---       ┆ ---   ┆   ┆ ---        ┆ e_date        ┆ se            ┆ ---          │
│ u32   ┆ u32  ┆ u32       ┆ u32   ┆   ┆ u32        ┆ ---           ┆ ---           ┆ u32          │
│       ┆      ┆           ┆       ┆   ┆            ┆ u32           ┆ u32           ┆              │
╞═══════╪══════╪═══════════╪═══════╪═══╪════════════╪═══════════════╪═══════════════╪══════════════╡
│ 0     ┆ 0    ┆ 0         ┆ 0     ┆ … ┆ 0          ┆ 0             ┆ 265657        ┆ 0            │
└───────┴──────┴───────────┴───────┴───┴────────────┴───────────────┴───────────────┴──────────────┘


In [7]:
# Calculate months from earliest month in dataset (Mar-2012)
df_combined_col = df_combined_col.with_columns(
    flat_age_years = (pl.col("month").dt.year() - pl.col("lease_commence_date")),
    days_from_earliest_data = (pl.col('month') - pl.date(2012, 3, 1)).dt.total_days()
).drop("remaining_lease") # can be dropped

In [8]:
# Checking unique values of towns
pl.Config.set_tbl_rows(100)
df_combined_col.select(pl.col("town").unique())

town
str
"""KALLANG/WHAMPOA"""
"""WOODLANDS"""
"""JURONG EAST"""
"""SERANGOON"""
"""PASIR RIS"""
"""PUNGGOL"""
"""ANG MO KIO"""
"""BUKIT BATOK"""
"""TAMPINES"""


In [9]:
# Check flat type
df_combined_col.select(pl.col("flat_type").unique())
#df_combined_col.select(pl.col("block").unique())

flat_type
str
"""1 ROOM"""
"""2 ROOM"""
"""MULTI-GENERATION"""
"""EXECUTIVE"""
"""5 ROOM"""
"""4 ROOM"""
"""3 ROOM"""


In [10]:
# Check flat model futher, by year
df_combined_col.group_by("lease_commence_date").agg(
    pl.col("flat_model").unique().alias("unique_flat_models"),
    pl.col("flat_type").unique().alias("unique_flat_types")
).sort("lease_commence_date").head(20)

df_combined_col.group_by("flat_model").agg(
    pl.col("lease_commence_date").min().alias("min_lease_commence_date"),
    pl.col("lease_commence_date").max().alias("max_lease_commence_date"),
    pl.col("lease_commence_date").count().alias("counts"),
).sort("max_lease_commence_date")

# df_combined_col.group_by("flat_type").agg(
#     pl.col("lease_commence_date").min().alias("min_lease_commence_date"),
#     pl.col("lease_commence_date").max().alias("max_lease_commence_date"),
#     pl.col("lease_commence_date").count().alias("counts"),
# ).sort("max_lease_commence_date")


# df_combined_col.with_columns(year=pl.col("month").dt.year()).group_by("year").agg(
#     pl.col("flat_model").unique().alias("unique_flat_models")
# ).sort("year").head(20)



flat_model,min_lease_commence_date,max_lease_commence_date,counts
str,i64,i64,u32
"""Terrace""",1968,1972,169
"""Improved-Maisonette""",1983,1983,40
"""Standard""",1967,1984,9035
"""Model A-Maisonette""",1983,1985,519
"""Multi Generation""",1987,1988,110
"""Adjoined flat""",1967,1989,497
"""Premium Maisonette""",1997,1998,23
"""New Generation""",1976,1999,42128
"""Maisonette""",1984,2000,8399


In [11]:
# Cleaning flat model: Combine Maisonettes
df_combined_col =df_combined_col.with_columns(flat_model_revised = pl.when(pl.col("flat_model").str.contains("Maisonette"))
    .then(pl.lit("Maisonette"))
    .otherwise(pl.col("flat_model"))
)
df_combined_col.select(pl.col("flat_model_revised").unique()).sort("flat_model_revised")

flat_model_revised
str
"""2-room"""
"""3Gen"""
"""Adjoined flat"""
"""Apartment"""
"""DBSS"""
"""Improved"""
"""Maisonette"""
"""Model A"""
"""Model A2"""


In [12]:
# Checking unique values of storey blocks
pl.Config.set_tbl_rows(100)
df_combined_col.select(pl.col("storey_range").unique())

storey_range
str
"""21 TO 25"""
"""36 TO 40"""
"""07 TO 09"""
"""01 TO 03"""
"""19 TO 21"""
"""25 TO 27"""
"""04 TO 06"""
"""22 TO 24"""
"""46 TO 48"""


In [13]:
# Create min max value for stories, then group every 15 storeys since a part of the data groups every 5 floors instead of 3 floors
df_combined_col = df_combined_col.with_columns(
    storey_max = pl.col("storey_range").str.slice(-2,2).cast(pl.Int64)
)


df_combined_col = df_combined_col.with_columns(
    storey_range_grouped = (pl.when(pl.col("storey_max")<= 15).then(pl.lit("0-15"))
                            .when(pl.col("storey_max").is_between(16, 30)).then(pl.lit("16-30"))
                            .otherwise(pl.lit("31+")))
)
df_combined_col.group_by("storey_range_grouped").agg(
    pl.col("storey_max").min().alias("min_storey"),
    pl.col("storey_max").max().alias("max_storey"),
    pl.col("storey_max").count().alias("counts")
).sort("storey_range_grouped")

storey_range_grouped,min_storey,max_storey,counts
str,i64,i64,u32
"""0-15""",3,15,275412
"""16-30""",18,30,25067
"""31+""",33,51,2331


In [14]:
# Other factors to KIV: Distance from MRT/amenities, lease years left, supply and demand of surrounding areas

In [16]:
df_combined_col.columns

['month',
 'town',
 'flat_type',
 'block',
 'street_name',
 'storey_range',
 'floor_area_sqm',
 'flat_model',
 'lease_commence_date',
 'resale_price',
 'flat_age_years',
 'days_from_earliest_data',
 'flat_model_revised',
 'storey_max',
 'storey_range_grouped']

In [17]:
# Keep relevant columns
df_output = df_combined_col.select(
        'month',
        'town',
        'flat_type',
        'flat_model_revised',
        'flat_age_years',
        'floor_area_sqm',
        'days_from_earliest_data',
        'resale_price',
)

# Check for nulls
null_counts = df_output.null_count()
print("Null counts in each column:")
print(null_counts)

Null counts in each column:
shape: (1, 8)
┌───────┬──────┬───────────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┐
│ month ┆ town ┆ flat_type ┆ flat_model_r ┆ flat_age_yea ┆ floor_area_ ┆ days_from_e ┆ resale_pric │
│ ---   ┆ ---  ┆ ---       ┆ evised       ┆ rs           ┆ sqm         ┆ arliest_dat ┆ e           │
│ u32   ┆ u32  ┆ u32       ┆ ---          ┆ ---          ┆ ---         ┆ a           ┆ ---         │
│       ┆      ┆           ┆ u32          ┆ u32          ┆ u32         ┆ ---         ┆ u32         │
│       ┆      ┆           ┆              ┆              ┆             ┆ u32         ┆             │
╞═══════╪══════╪═══════════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╡
│ 0     ┆ 0    ┆ 0         ┆ 0            ┆ 0            ┆ 0           ┆ 0           ┆ 0           │
└───────┴──────┴───────────┴──────────────┴──────────────┴─────────────┴─────────────┴─────────────┘


In [ ]:
# Split into train (2012-2023), test(2024) and deploy sets (2025)
train = df_output.filter(df_output['month'].dt.year().is_between(2012, 2023)).drop("month").write_parquet("../data/processed/train.parquet")
test = df_output.filter(df_output['month'].dt.year() == 2024).drop("month").write_parquet("../data/processed/test.parquet")
deploy = df_output.filter(df_output['month'].dt.year() == 2025).drop("month").write_parquet("../data/processed/deploy.parquet")